In [9]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, roc_auc_score, plot_roc_curve
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import plot_roc_curve
from mlxtend.plotting import plot_decision_regions
from sklearn.datasets import make_classification
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import learning_curve

from lightgbm import LGBMClassifier
from lightgbm import plot_importance

from sklearn.metrics import confusion_matrix, plot_confusion_matrix, precision_score, recall_score, plot_roc_curve
from sklearn.metrics import classification_report


In [49]:
rw = pd.read_csv('./data/rw_clean.csv', index_col=0)

In [57]:
import datetime

In [66]:
rw['date']=pd.to_datetime(rw['date'])

In [71]:
dia=((rw['date'])[1] - datetime.timedelta(days=1))

In [72]:
rw.loc[rw['date']==dia,:]

,date,location,rainfall,humidity3pm,pressure3pm,temp3pm,raintomorrow,modelo_vigente,wind_dir3pm,wind_speed3pm,...,SE,SSE,SSW,SW,W,WNW,WSW,cluster_1,cluster_2,cluster_3
0,2008-12-01,Albury,0.6,22.0,1007.1,21.8,0,0.089825,WNW,24.0,...,0,0,0,0,0,1,0,1,0,0
26181,2008-12-01,Sydney,0.0,21.0,1004.6,30.8,0,0.030881,WNW,24.0,...,0,0,0,0,0,1,0,1,0,2
42924,2008-12-01,Canberra,0.0,28.0,1004.4,23.9,0,0.021746,NW,43.0,...,0,0,0,0,0,0,0,1,0,2
46147,2008-12-01,Tuggeranong,0.0,34.0,1003.6,23.1,0,0.027801,WNW,33.0,...,0,0,0,0,0,1,0,1,0,2
49510,2008-12-01,Ballarat,3.2,37.0,1007.5,15.7,0,0.093167,WNW,30.0,...,0,0,0,0,0,1,0,1,0,0
52946,2008-12-01,Bendigo,0.0,23.0,1007.5,19.4,0,0.031342,WSW,20.0,...,0,0,0,0,0,0,1,1,0,0
63370,2008-12-01,Melbourne,1.4,32.0,1007.8,19.3,0,0.064315,W,31.0,...,0,0,0,0,1,0,0,1,0,0
81569,2008-12-01,Brisbane,0.0,41.0,1010.1,28.6,0,0.004988,NE,13.0,...,0,0,0,0,0,0,0,1,0,1
84967,2008-12-01,Cairns,0.4,66.0,1008.1,30.7,1,0.633994,ENE,13.0,...,0,0,0,0,0,0,0,2,3,1
88321,2008-12-01,GoldCoast,0.0,61.0,1010.7,28.5,0,0.063194,NE,22.0,...,0,0,0,0,0,0,0,1,0,1


In [26]:
rw = pd.read_csv('./data/rw_clean.csv', index_col=0)
rw=rw.drop(columns=['date','wind_dir3pm','modelo_vigente','location'])
df_aux=[]
lgbm={}
logistic={}
tree = {}
for x in range(1,4):
    for i in range(0,rw[f'cluster_{x}'].unique().max()+1):
        df_aux =rw[(rw[f'cluster_{x}'] == i)]
        df_aux = df_aux.drop(['lat', 'lng', 'cluster_1', 'cluster_2', 'cluster_3'], axis=1)
        y = df_aux.raintomorrow
        X = df_aux.drop('raintomorrow', axis=1)
        lgb = LGBMClassifier(max_depth=5, num_leaves=32, min_data_in_leaf=6, 
                     learning_rate=0.001, n_estimators=1000, 
                     bagging_fraction=1, bagging_freq=2, feature_fraction=1,
                     max_bin=5,
                     colsample_bytree=.7, n_jobs=-1)
        results = cross_val_score(lgb, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
        name=f'modelo {x} cluster {i}'
        lgbm[name]=(np.mean(results), np.std(results))
        log = LogisticRegression(n_jobs=-1)
        results = cross_val_score(log, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
        logistic[name]=(np.mean(results), np.std(results))
        tree_m = DecisionTreeClassifier(max_depth=5,ccp_alpha=0.001,random_state=42)
        results = cross_val_score(tree_m, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
        tree[name]=(np.mean(results), np.std(results))
        

In [41]:
rw = pd.read_csv('./data/rw_clean.csv', index_col=0)
rw=rw.drop(columns=['date','wind_dir3pm','modelo_vigente','location', 'cluster_1', 'cluster_2', 'cluster_3'])
y = rw.raintomorrow
X = rw.drop('raintomorrow', axis=1)
lgb = LGBMClassifier(max_depth=5, num_leaves=32, min_data_in_leaf=6, 
             learning_rate=0.001, n_estimators=1000, 
             bagging_fraction=1, bagging_freq=2, feature_fraction=1,
             max_bin=5,
             colsample_bytree=.7, n_jobs=-1)
results = cross_val_score(lgb, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
lgbm=(np.mean(results), np.std(results))
log = LogisticRegression(n_jobs=-1)
results = cross_val_score(log, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
logistic=(np.mean(results), np.std(results))
tree_m = DecisionTreeClassifier(max_depth=5,ccp_alpha=0.001,random_state=42)
results = cross_val_score(tree_m, X, y, cv=KFold(4, shuffle=True), scoring='roc_auc')
tree=(np.mean(results), np.std(results))

In [36]:
pd.DataFrame(lgbm).T[0].mean()

0.8558101894502887

In [42]:
lgbm

(0.8432055150977585, 0.0039367251716415155)

In [37]:
pd.DataFrame(logistic).T[0].mean()

0.8451167838432551

In [43]:
logistic

(0.8208304543811356, 0.005815462783598415)

In [38]:
pd.DataFrame(tree).T[0].mean()

0.8273531029530695

In [44]:
tree

(0.8121806322900681, 0.0018434463627228811)

In [47]:
rain = pd.read_csv('./data/rain_data_aus.csv')